In [10]:


import numpy as np

class Linear:
    
    def __init__(self, in_features, out_features):
        self.weights = np.random.randn(in_features, out_features) * 0.01
        self.bias = np.zeros(out_features)
        self.input = None
        self.output = None

    def forward(self, x):
        self.input = x
        self.output = np.dot(x, self.weights) + self.bias
        return self.output

    def backward(self, d_output):
        d_input = np.dot(d_output, self.weights.T)
        d_weights = np.dot(self.input.T, d_output)
        d_bias = np.sum(d_output, axis=0)
        self.weights -= learning_rate * d_weights
        self.bias -= learning_rate * d_bias
        return d_input

class ReLU:
    def forward(self, x):
        self.input = x
        return np.maximum(0, x)

    def backward(self, d_output):
        return d_output * (self.input > 0)

class Sigmoid:
    def forward(self, x):
        self.input = x
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, d_output):
        return d_output * (self.output * (1 - self.output))

class Tanh:
    def forward(self, x):
        self.input = x
        self.output = np.tanh(x)
        return self.output

    def backward(self, d_output):
        return d_output * (1 - np.power(self.output, 2))

class Softmax:
    def forward(self, x):
        self.input = x
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return self.output

    def backward(self, d_output):
        return d_output

class CrossEntropyLoss:
    def forward(self, predictions, targets):
        self.predictions = predictions
        self.targets = targets
        return -np.sum(targets * np.log(predictions + 1e-9)) / predictions.shape[0]

    def backward(self):
        return (self.predictions - self.targets) / self.targets.shape[0]

class MSELoss:
    def forward(self, predictions, targets):
        self.predictions = predictions
        self.targets = targets
        return np.mean((predictions - targets) ** 2)

    def backward(self):
        return 2 * (self.predictions - self.targets) / self.targets.size

class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def step(self, layer):
        layer.weights -= self.learning_rate * layer.d_weights
        layer.bias -= self.learning_rate * layer.d_bias

class Model:
    def __init__(self):
        self.layers = []
        self.loss_function = None
        self.optimizer = None

    def add_layer(self, layer):
        self.layers.append(layer)

    def compile(self, loss_function, optimizer):
        self.loss_function = loss_function
        self.optimizer = optimizer

    def forward(self, x):
        self.activations = []
        self.inputs = [x]
        for layer in self.layers:
            x = layer.forward(x)
            self.inputs.append(x)
            self.activations.append(layer)
        return x

    def backward(self, d_output):
        for layer in reversed(self.activations):
            d_output = layer.backward(d_output)
            if isinstance(layer, Linear):
                self.optimizer.step(layer)

    def train(self, x_train, y_train, epochs, batch_size):
        for epoch in range(epochs):
            for start in range(0, x_train.shape[0], batch_size):
                end = start + batch_size
                x_batch, y_batch = x_train[start:end], y_train[start:end]
                predictions = self.forward(x_batch)
                loss = self.loss_function.forward(predictions, y_batch)
                d_loss = self.loss_function.backward()
                self.backward(d_loss)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss}')

    def evaluate(self, x_test, y_test):
        predictions = self.forward(x_test)
        loss = self.loss_function.forward(predictions, y_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
        return loss, accuracy

    def save(self, path):
        np.savez(path, weights=[layer.weights for layer in self.layers if isinstance(layer, Linear)],
                 bias=[layer.bias for layer in self.layers if isinstance(layer, Linear)])

    def load(self, path):
        data = np.load(path)
        weights = data['weights']
        bias = data['bias']
        idx = 0
        for layer in self.layers:
            if isinstance(layer, Linear):
                layer.weights = weights[idx]
                layer.bias = bias[idx]
                idx += 1


In [11]:
model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

# Compile the model with loss and optimizer
loss = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.01)
model.compile(loss, optimizer)

# Assume x_train, y_train, x_test, y_test are preprocessed and available
# Train the model
model.train(x_train, y_train, epochs=20, batch_size=64)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

NameError: name 'learning_rate' is not defined